In [15]:
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
import awswrangler as wr
import boto3
import os
import io


In [22]:
sess = sagemaker.Session()
role = get_execution_role()

In [37]:
prefix = 'rawdata/original_raw_files'
bucket = "humana-data"

conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=prefix)['Contents']

rawfile_names = [key['Key'] for key in contents]
rawfile_names = [x for x in rawfile_names if x.endswith('.csv')]
rawfile_names

['rawdata/original_raw_files/CMS.csv',
 'rawdata/original_raw_files/Condition.csv',
 'rawdata/original_raw_files/Credit.csv',
 'rawdata/original_raw_files/Demo.csv',
 'rawdata/original_raw_files/Lab.csv',
 'rawdata/original_raw_files/Medical Claims.csv',
 'rawdata/original_raw_files/Not sure.csv',
 'rawdata/original_raw_files/Others.csv',
 'rawdata/original_raw_files/Pharm.csv',
 'rawdata/original_raw_files/dependent.csv']

In [8]:
# Get the demo file
response = conn.get_object(Bucket=bucket, Key='rawdata/Demo.csv')
status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

if status == 200:
    print(f"Successful S3 get_object response. Status - {status}")
    df = pd.read_csv(response.get("Body"))
else:
    print(f"Unsuccessful S3 get_object response. Status - {status}")

Successful S3 get_object response. Status - 200


In [ ]:
#drop the column
df.drop(columns='transportation_issues')

In [98]:
#Export the file
with io.StringIO() as csv_buffer:
    df.to_csv(csv_buffer, index=False)

    response = s3_client.put_object(
        Bucket=bucket, Key="rawdata/Demo.csv", Body=csv_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response. Status - {status}")

Successful S3 put_object response. Status - 200


### Export files as Parquet

In [34]:
prefix_pq = prefix + '/original_pq_files'

In [50]:
df = wr.s3.read_parquet(path = f's3://{bucket}/{prefix_pq}/Demo.pq')

wr.s3.to_parquet(df,
                path = f's3://{bucket}/{prefix_pq}/Demo.pq', compression='gzip')

In [48]:
#IMPORT as csv and export as parquet file format
for file in rawfile_names:
    filename_pq = file.split('/')[-1].split('.')[0] + '.pq'
    
    # Get the demo file
    response = conn.get_object(Bucket=bucket, Key=file)
    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 get_object response. Status - {status}")
        df = pd.read_csv(response.get("Body"))
        wr.s3.to_parquet(df,
                path = f's3://{bucket}/{prefix_pq}/{filename_pq}', compression='gzip')
        
    else:
        print(f"Unsuccessful S3 get_object response. Status - {status}")

Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200


In [54]:
bucket

'humana-data'

In [56]:
prefix

'rawdata/original_raw_files'

In [58]:
response = conn.get_object(Bucket=bucket, Key=prefix)

NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [66]:
prefix

'rawdata/original_raw_files'

In [70]:
response = conn.get_object(Bucket=bucket, Key=prefix+'/Credit.csv')
status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")


if status == 200:
    print(f"Successful S3 get_object response. Status - {status}")
    df = pd.read_csv(response.get("Body")
                    )
wr.s3.to_parquet(df,
                path = f's3://{bucket}/{prefix_pq}/Credit.pq', compression='gzip')

Successful S3 get_object response. Status - 200


{'paths': ['s3://humana-data/rawdata/original_pq_files/Credit.pq'],
 'partitions_values': {}}